In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

#libararies
import pandas as pd


In [5]:
drive.mount('/content/gdrive')
path='/content/gdrive/My Drive/GP | Smart Meter | CIC/Datasets/Smart meters in London'

Mounted at /content/gdrive


In [6]:
for num in range(0,112):
  df = pd.read_csv(path + "/daily_dataset/daily_dataset/block_" + str(num) + ".csv")
  df = df[['day','LCLid','energy_sum']]
  df.reset_index()
  df.to_csv("hc_"+str(num)+".csv")

fout= open("energy.csv","a")
# first file:
for line in open("hc_0.csv"):
  fout.write(line)
# now the rest:    
for num in range(0,112):
  f = open("hc_"+str(num)+".csv")
  f.readline() # skip the header
  for line in f:
      fout.write(line)
  f.close()
fout.close()

In [7]:
energy = pd.read_csv('energy.csv')
len(energy)

3536007

In [8]:
energy.shape  

(3536007, 4)

In [9]:
#energy.head()

In [10]:
energy.drop('Unnamed: 0',axis='columns', inplace=True)

In [11]:
#energy.head()

In [12]:
#energy.shape

In [13]:
energy.day = pd.to_datetime(energy.day,format='%Y-%m-%d').dt.date

In [14]:
energy.head()

,day,LCLid,energy_sum
0,2012-10-12,MAC000002,7.098
1,2012-10-13,MAC000002,11.087
2,2012-10-14,MAC000002,13.223
3,2012-10-15,MAC000002,10.257
4,2012-10-16,MAC000002,9.769


In [15]:
energy.shape

(3536007, 3)

In [16]:
#housecount = energy.groupby('day')[['LCLid']].nunique()
#housecount.head(10)

In [17]:
#housecount.plot(figsize=(25,5))


# Processing over weather file

In [18]:
weather = pd.read_csv(path + "/weather_daily_darksky.csv")
weather.head(2)

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,10.87,rain,3.30,0.95,2011-11-11 19:00:00,10.87,11.96,1.0,2011-11-11 00:00:00,2011-11-11 16:19:21,10.87,8.85,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,5.62,rain,12.09,0.88,2011-12-11 19:00:00,-0.64,5.72,1.0,2011-12-11 00:00:00,2011-12-11 15:52:53,3.09,2.48,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53


In [19]:
#weather.describe()

In [20]:
weather.precipType.unique()

array(['rain', 'snow'], dtype=object)

In [21]:
weather["precipType"].replace({"rain": 1, "snow": 0}, inplace=True)

In [22]:
#weather.head()

In [23]:
weather.icon.unique()

array(['fog', 'partly-cloudy-day', 'wind', 'partly-cloudy-night',
       'clear-day', 'cloudy'], dtype=object)

In [24]:
from sklearn.preprocessing import OneHotEncoder
icon_one_hot_decoded = pd.get_dummies(weather.icon,prefix='icon')

In [25]:
icon_one_hot_decoded.head()

,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,0,0,1
4,0,0,0,0,1,0


In [26]:
weather = pd.concat([weather, icon_one_hot_decoded], axis=1)


In [27]:
weather.head()

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,10.87,1,3.30,0.95,2011-11-11 19:00:00,10.87,11.96,1.0,2011-11-11 00:00:00,2011-11-11 16:19:21,10.87,8.85,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52,0,0,1,0,0,0
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,5.62,1,12.09,0.88,2011-12-11 19:00:00,-0.64,5.72,1.0,2011-12-11 00:00:00,2011-12-11 15:52:53,3.09,2.48,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53,0,0,0,1,0,0
2,10.33,2011-12-27 02:00:00,225,partly-cloudy-day,5.47,2011-12-27 23:00:00,0.85,3.54,1032.76,2011-12-27 22:00:00,10.33,1,13.39,0.74,2011-12-27 14:00:00,5.52,10.33,0.0,2011-12-27 00:00:00,2011-12-27 15:57:56,8.03,8.03,10.33,2011-12-27 08:07:06,2011-12-27 14:00:00,2011-12-27 00:00:00,Mostly cloudy throughout the day.,2011-12-27 23:00:00,5.59,2011-12-27 02:00:00,2011-12-28 00:00:00,0.10,0,0,0,1,0,0
3,8.07,2011-12-02 23:00:00,232,wind,3.69,2011-12-02 07:00:00,0.32,3.00,1012.12,2011-12-02 07:00:00,5.33,1,11.89,0.87,2011-12-02 12:00:00,3.26,5.33,1.0,2011-12-02 00:00:00,2011-12-02 15:56:17,6.33,2.56,7.36,2011-12-02 07:46:09,2011-12-02 12:00:00,2011-12-02 10:00:00,Partly cloudy throughout the day and breezy ov...,2011-12-02 19:00:00,0.46,2011-12-02 12:00:00,2011-12-02 19:00:00,0.25,0,0,0,0,0,1
4,8.22,2011-12-24 23:00:00,252,partly-cloudy-night,2.79,2011-12-24 07:00:00,0.37,4.46,1028.17,2011-12-24 07:00:00,5.02,1,13.16,0.80,2011-12-24 15:00:00,4.37,5.32,1.0,2011-12-24 00:00:00,2011-12-24 15:55:55,7.45,3.17,7.93,2011-12-24 08:06:15,2011-12-24 15:00:00,2011-12-24 13:00:00,Mostly cloudy throughout the day.,2011-12-24 19:00:00,-0.51,2011-12-24 23:00:00,2011-12-24 20:00:00,0.99,0,0,0,0,1,0


In [28]:
weather.summary.unique()

array(['Foggy until afternoon.', 'Partly cloudy throughout the day.',
       'Mostly cloudy throughout the day.',
       'Partly cloudy throughout the day and breezy overnight.',
       'Partly cloudy throughout the day and breezy in the morning.',
       'Foggy starting in the evening.',
       'Partly cloudy throughout the day and breezy starting in the evening.',
       'Foggy in the morning.', 'Partly cloudy until evening.',
       'Partly cloudy starting in the afternoon, continuing until evening.',
       'Breezy in the morning and partly cloudy throughout the day.',
       'Partly cloudy throughout the day and breezy until afternoon.',
       'Breezy and mostly cloudy throughout the day.', 'Foggy overnight.',
       'Mostly cloudy throughout the day and breezy overnight.',
       'Partly cloudy until afternoon.',
       'Mostly cloudy until evening and breezy starting in the afternoon, continuing until evening.',
       'Breezy in the morning and partly cloudy until afternoon.',

In [29]:
weather['day']=  pd.to_datetime(weather['time']) # day is given as timestamp


In [30]:
weather.head()

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,day
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,10.87,1,3.30,0.95,2011-11-11 19:00:00,10.87,11.96,1.0,2011-11-11 00:00:00,2011-11-11 16:19:21,10.87,8.85,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52,0,0,1,0,0,0,2011-11-11
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,5.62,1,12.09,0.88,2011-12-11 19:00:00,-0.64,5.72,1.0,2011-12-11 00:00:00,2011-12-11 15:52:53,3.09,2.48,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53,0,0,0,1,0,0,2011-12-11
2,10.33,2011-12-27 02:00:00,225,partly-cloudy-day,5.47,2011-12-27 23:00:00,0.85,3.54,1032.76,2011-12-27 22:00:00,10.33,1,13.39,0.74,2011-12-27 14:00:00,5.52,10.33,0.0,2011-12-27 00:00:00,2011-12-27 15:57:56,8.03,8.03,10.33,2011-12-27 08:07:06,2011-12-27 14:00:00,2011-12-27 00:00:00,Mostly cloudy throughout the day.,2011-12-27 23:00:00,5.59,2011-12-27 02:00:00,2011-12-28 00:00:00,0.10,0,0,0,1,0,0,2011-12-27
3,8.07,2011-12-02 23:00:00,232,wind,3.69,2011-12-02 07:00:00,0.32,3.00,1012.12,2011-12-02 07:00:00,5.33,1,11.89,0.87,2011-12-02 12:00:00,3.26,5.33,1.0,2011-12-02 00:00:00,2011-12-02 15:56:17,6.33,2.56,7.36,2011-12-02 07:46:09,2011-12-02 12:00:00,2011-12-02 10:00:00,Partly cloudy throughout the day and breezy ov...,2011-12-02 19:00:00,0.46,2011-12-02 12:00:00,2011-12-02 19:00:00,0.25,0,0,0,0,0,1,2011-12-02
4,8.22,2011-12-24 23:00:00,252,partly-cloudy-night,2.79,2011-12-24 07:00:00,0.37,4.46,1028.17,2011-12-24 07:00:00,5.02,1,13.16,0.80,2011-12-24 15:00:00,4.37,5.32,1.0,2011-12-24 00:00:00,2011-12-24 15:55:55,7.45,3.17,7.93,2011-12-24 08:06:15,2011-12-24 15:00:00,2011-12-24 13:00:00,Mostly cloudy throughout the day.,2011-12-24 19:00:00,-0.51,2011-12-24 23:00:00,2011-12-24 20:00:00,0.99,0,0,0,0,1,0,2011-12-24


In [31]:
weather['day']=  pd.to_datetime(weather['day'],format='%Y%m%d').dt.date
weather.head(2)

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,day
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,10.87,1,3.30,0.95,2011-11-11 19:00:00,10.87,11.96,1.0,2011-11-11 00:00:00,2011-11-11 16:19:21,10.87,8.85,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52,0,0,1,0,0,0,2011-11-11
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,5.62,1,12.09,0.88,2011-12-11 19:00:00,-0.64,5.72,1.0,2011-12-11 00:00:00,2011-12-11 15:52:53,3.09,2.48,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53,0,0,0,1,0,0,2011-12-11


In [32]:
# selecting numeric variables
weather = weather[['temperatureMax', 'windBearing', 'dewPoint', 'cloudCover', 'windSpeed',
       'pressure', 'apparentTemperatureHigh','precipType', 'visibility', 'humidity',
       'apparentTemperatureLow', 'apparentTemperatureMax', 'uvIndex',
       'temperatureLow', 'temperatureMin', 'temperatureHigh',
       'apparentTemperatureMin', 'moonPhase','icon_clear-day','icon_cloudy','icon_fog','icon_partly-cloudy-day','icon_partly-cloudy-night','icon_wind','day']]

In [33]:
weather = weather.dropna()


In [34]:
weather.head()

,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,day
0,11.96,123,9.40,0.79,3.88,1016.08,10.87,1,3.30,0.95,10.87,11.96,1.0,10.87,8.85,10.87,6.48,0.52,0,0,1,0,0,0,2011-11-11
1,8.59,198,4.49,0.56,3.94,1007.71,5.62,1,12.09,0.88,-0.64,5.72,1.0,3.09,2.48,8.59,0.11,0.53,0,0,0,1,0,0,2011-12-11
2,10.33,225,5.47,0.85,3.54,1032.76,10.33,1,13.39,0.74,5.52,10.33,0.0,8.03,8.03,10.33,5.59,0.10,0,0,0,1,0,0,2011-12-27
3,8.07,232,3.69,0.32,3.00,1012.12,5.33,1,11.89,0.87,3.26,5.33,1.0,6.33,2.56,7.36,0.46,0.25,0,0,0,0,0,1,2011-12-02
4,8.22,252,2.79,0.37,4.46,1028.17,5.02,1,13.16,0.80,4.37,5.32,1.0,7.45,3.17,7.93,-0.51,0.99,0,0,0,0,1,0,2011-12-24


In [35]:
weather.shape

(881, 25)

#Merge Energy Dataframe with weather Dataframe

In [36]:
weather_energy =  energy.merge(weather,on='day')
weather_energy.head(2)

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind
0,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0
1,2012-10-12,MAC000246,17.245,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0


In [37]:
weather_energy.shape

(3526707, 27)

Note:<br>
data = 3536007 - 3526707 <br>
lost 9300

#Household Data

In [38]:
informations_households = pd.read_csv(path + "/informations_households.csv")
informations_households.head(20)

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0
2,MAC000002,Std,ACORN-A,Affluent,block_0
3,MAC003613,Std,ACORN-A,Affluent,block_0
4,MAC003597,Std,ACORN-A,Affluent,block_0
5,MAC003579,Std,ACORN-A,Affluent,block_0
6,MAC003566,Std,ACORN-A,Affluent,block_0
7,MAC003557,Std,ACORN-A,Affluent,block_0
8,MAC003553,Std,ACORN-A,Affluent,block_0
9,MAC003482,Std,ACORN-A,Affluent,block_0


In [39]:
informations_households.stdorToU.unique()

array(['ToU', 'Std'], dtype=object)

In [40]:
informations_households["stdorToU"].replace({"ToU": 1, "Std": 0}, inplace=True)

In [41]:
informations_households.Acorn.unique() #first trial without it without it

array(['ACORN-', 'ACORN-A', 'ACORN-B', 'ACORN-C', 'ACORN-D', 'ACORN-E',
       'ACORN-F', 'ACORN-G', 'ACORN-H', 'ACORN-I', 'ACORN-J', 'ACORN-K',
       'ACORN-L', 'ACORN-M', 'ACORN-N', 'ACORN-O', 'ACORN-P', 'ACORN-Q',
       'ACORN-U'], dtype=object)

In [42]:
informations_households.Acorn_grouped.unique()

array(['ACORN-', 'Affluent', 'Comfortable', 'Adversity', 'ACORN-U'],
      dtype=object)

In [43]:
#Acorn_one_hot_decoded = pd.get_dummies(informations_households.Acorn_grouped,prefix='Acorn')

In [44]:
#Acorn_one_hot_decoded.head()

#Try Label Encoding 

In [45]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
informations_households['Acorn_grouped'] = label_encoder.fit_transform(informations_households['Acorn_grouped'])
informations_households['file'] = label_encoder.fit_transform(informations_households['file'])


In [46]:
informations_households.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,1,ACORN-,0,0
1,MAC001074,1,ACORN-,0,0
2,MAC000002,0,ACORN-A,3,0
3,MAC003613,0,ACORN-A,3,0
4,MAC003597,0,ACORN-A,3,0


In [47]:
#informations_households = pd.concat([informations_households, Acorn_one_hot_decoded], axis=1)


In [48]:
#informations_households = informations_households.rename(columns={"file": "block"})


In [49]:
#informations_households.block.unique()

In [50]:
#block_one_hot_decoded = pd.get_dummies(informations_households.block)

In [51]:
#block_one_hot_decoded.head()

In [52]:
#informations_households['block'].astype('category')

In [53]:
#informations_households['block'] = informations_households['block'].cat.codes

In [54]:
#informations_households = pd.concat([informations_households, block_one_hot_decoded], axis=1)


In [55]:
#informations_households.head()

In [56]:
informations_households = informations_households.drop(['Acorn'], axis=1)




#Merge household data to dataset

In [57]:
data =  weather_energy.merge(informations_households,on='LCLid')


In [58]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file
0,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0


In [59]:
data.shape

(3526707, 30)

In [60]:
#weather.shape

In [61]:
#weather.head()

# Holiday Data


In [62]:
holiday = pd.read_csv(path + "/uk_bank_holidays.csv")


In [63]:
holiday.head()

,Bank holidays,Type
0,2012-12-26,Boxing Day
1,2012-12-25,Christmas Day
2,2012-08-27,Summer bank holiday
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday)
4,2012-04-06,Spring bank holiday (substitute day)


In [64]:
#uk_bank_holidays = uk_bank_holidays.rename(columns={"Bank holidays": "day"})
holiday['Bank holidays'] = pd.to_datetime(holiday['Bank holidays'],format='%Y-%m-%d').dt.date

In [65]:
holiday.head(4)


,Bank holidays,Type
0,2012-12-26,Boxing Day
1,2012-12-25,Christmas Day
2,2012-08-27,Summer bank holiday
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday)


In [66]:
#uk_bank_holidays.day = pd.to_datetime(uk_bank_holidays.day ,format='%Y-%m-%d').dt.date

In [67]:
#uk_bank_holidays.day.unique()

In [68]:
#uk_bank_holidays['Type'] = 1

In [69]:
#uk_bank_holidays.head()

In [70]:
#uk_bank_holidays.shape

#Merge holidays

In [71]:
import numpy as np
data = data.merge(holiday, left_on = 'day',right_on = 'Bank holidays',how = 'left')
data['holiday_ind'] = np.where(data['Bank holidays'].isna(),0,1)

In [72]:
data.shape

(3526707, 33)

In [73]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,Bank holidays,Type,holiday_ind
0,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,NaN,NaN,0
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,NaN,NaN,0
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,NaN,NaN,0
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,NaN,NaN,0
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,NaN,NaN,0


In [74]:
data.holiday_ind.unique()

array([0, 1])

In [75]:
#if data['day'].values == uk_bank_holidays['day'].values:
#  data['holiday'] = 1
#else:
#  data['holiday'] = 0

In [76]:
data = data.drop(['Bank holidays','Type'], axis = 1)

In [77]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind
0,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0


#Create Last day's consumption

In [78]:
data['last_day_sum'] = data['energy_sum'].shift()
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum
0,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0,NaN
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0,7.098
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,7.098
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,11.087
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,11.087


In [79]:
data = data.dropna()


In [80]:
data.shape

(3526650, 32)

**After** removing nan values data lost 57 raws total of 93057

#Add Season Column

In [81]:
data['day'] = pd.to_datetime(data.day, format='%Y-%m-%d %H:%M:%S')


In [82]:
data['month'] = data['day'].dt.month 


In [83]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0,7.098,10
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,7.098,10
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,11.087,10
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,11.087,10
5,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,13.223,10


In [84]:
data.loc[(data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2) , 'season'] = 'winter'
data.loc[(data['month'] == 3) | (data['month'] == 4) | (data['month'] == 5) , 'season'] = 'spring'
data.loc[(data['month'] == 6) | (data['month'] == 7) | (data['month'] == 8) , 'season'] = 'summer'
data.loc[(data['month'] == 9) | (data['month'] == 10) | (data['month'] ==11) , 'season'] = 'autumn'

In [85]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0,7.098,10,autumn
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,7.098,10,autumn
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,11.087,10,autumn
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,11.087,10,autumn
5,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,13.223,10,autumn


In [86]:
data.season.unique()

array(['autumn', 'winter', 'spring', 'summer'], dtype=object)

In [87]:
data['season'] = label_encoder.fit_transform(data['season'])


In [88]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0,7.098,10,0
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,7.098,10,0
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,11.087,10,0
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,11.087,10,0
5,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,13.223,10,0


In [89]:
data.season.unique()

array([0, 3, 1, 2])

In [90]:
data.shape

(3526650, 34)

# Take only some data points **According to years**


In [91]:
data['year'] = data['day'].dt.year


In [92]:
data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season,year
1,2012-10-12,MAC000002,7.098,11.53,252,6.15,0.29,2.18,1004.92,11.53,1,12.94,0.84,1.64,11.53,2.0,2.81,6.41,11.53,4.01,0.92,0,0,0,1,0,0,0,3,0,0,7.098,10,0,2012
2,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,7.098,10,0,2012
3,2012-10-13,MAC000002,11.087,12.32,300,4.10,0.20,2.36,1002.50,12.32,1,11.86,0.80,2.83,12.32,2.0,4.09,2.81,12.32,1.64,0.96,0,0,0,1,0,0,0,3,0,0,11.087,10,0,2012
4,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,11.087,10,0,2012
5,2012-10-14,MAC000002,13.223,13.03,220,6.39,0.31,2.97,1002.67,13.03,1,12.17,0.84,7.15,13.03,1.0,9.81,4.09,13.03,2.83,0.99,0,0,0,1,0,0,0,3,0,0,13.223,10,0,2012


In [93]:
data.year.unique()

array([2012, 2013, 2014, 2011])

In [94]:

part_of_data_2013 = data[data["year"] == 2013]

In [95]:
part_of_data_2013.shape

(1957607, 35)

In [96]:
part_of_data_2011 = data[data["year"] == 2011]

In [97]:
part_of_data_2011.shape

(8552, 35)

In [98]:
part_of_data_2012 = data[data["year"] == 2012]

In [99]:
part_of_data_2012.shape

(1264991, 35)

In [100]:
part_of_data_2014 = data[data["year"] == 2014]

In [101]:
part_of_data_2014.shape

(295500, 35)

2011 =~ 8K <br>
2012 =~ 1.2M <br>
2013 =~ 1.9M <br>
2014 =~ 295K <br>

In [102]:
part_of_data_2011.season.unique()

array([3, 0])

In [103]:
part_of_data_2014.season.unique()

array([3])

# Take some datapoints according to **season**




1.   Each season alone
2.   shuffling 
3.   take random 10K from each season **To ensure varity specialy in years** 


In [104]:
season_0 = data.loc[data['season'] == 0]


In [105]:
season_0.season.unique()

array([0])

In [106]:
from sklearn.utils import shuffle
season_0 = shuffle(season_0)

In [107]:
season_1 = data.loc[data['season'] == 1]
season_2 = data.loc[data['season'] == 2]
season_3 = data.loc[data['season'] == 3]


In [108]:
season_1 = shuffle(season_1)
season_2 = shuffle(season_2)
season_3 = shuffle(season_3)

In [109]:
df0 = season_0.sample(n=10000)
df1 = season_1.sample(n=10000)
df2 = season_2.sample(n=10000)
df3 = season_3.sample(n=10000)


In [110]:

new_data = df0.append([ df1,df2,df3])

In [111]:
new_data.shape

(40000, 35)

In [112]:
new_data.season.unique()

array([0, 1, 2, 3])

In [113]:
new_data.year.unique()

array([2012, 2013, 2011, 2014])

In [114]:
new_data = shuffle(new_data)


In [115]:
new_data.head()

,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season,year
2175289,2012-07-19,MAC001497,4.364,17.77,340,11.44,0.54,1.52,1016.21,17.77,1,12.18,0.79,11.83,17.77,5.0,11.83,13.12,17.77,13.12,0.04,0,0,0,1,0,0,0,4,80,0,3.952,7,2,2012
2203521,2013-10-03,MAC003176,10.340,20.82,208,15.39,0.62,4.04,1009.01,20.84,1,11.68,0.87,13.67,20.84,1.0,13.67,15.23,20.82,15.23,0.98,0,0,0,1,0,0,0,4,81,0,13.006,10,0,2013
2158078,2012-09-07,MAC001679,4.236,27.12,215,9.27,0.14,1.39,1020.60,27.12,1,12.36,0.62,11.32,27.12,4.0,11.32,10.41,27.12,10.41,0.75,0,0,0,1,0,0,0,4,78,0,3.550,9,0,2012
130514,2013-10-26,MAC001544,13.980,15.87,225,10.34,0.32,7.05,996.66,15.87,1,11.18,0.81,11.70,15.87,1.0,11.70,11.66,15.87,11.66,0.76,0,0,0,0,0,1,0,3,46,0,23.422,10,0,2013
1508327,2013-03-15,MAC000655,6.663,8.27,207,2.72,0.65,5.82,1007.60,4.56,1,12.36,0.81,1.78,4.56,2.0,6.36,2.99,8.27,-0.32,0.12,0,0,0,0,0,1,0,4,54,0,7.472,3,1,2013


# Download the data

In [120]:
from google.colab import files
new_data.to_csv('sample_data.csv') 
files.download('sample_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Here**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd


In [6]:
drive.mount('/content/gdrive')
path='/content/gdrive/My Drive/GP | Smart Meter | CIC/Datasets'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
new_data = pd.read_csv(path + "/sample_data_.csv")


In [8]:
new_data.head()

,Unnamed: 0,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season,year
0,2175289,2012-07-19,MAC001497,4.364,17.77,340,11.44,0.54,1.52,1016.21,17.77,1,12.18,0.79,11.83,17.77,5.0,11.83,13.12,17.77,13.12,0.04,0,0,0,1,0,0,0,4,80,0,3.952,7,2,2012
1,2203521,2013-10-03,MAC003176,10.340,20.82,208,15.39,0.62,4.04,1009.01,20.84,1,11.68,0.87,13.67,20.84,1.0,13.67,15.23,20.82,15.23,0.98,0,0,0,1,0,0,0,4,81,0,13.006,10,0,2013
2,2158078,2012-09-07,MAC001679,4.236,27.12,215,9.27,0.14,1.39,1020.60,27.12,1,12.36,0.62,11.32,27.12,4.0,11.32,10.41,27.12,10.41,0.75,0,0,0,1,0,0,0,4,78,0,3.550,9,0,2012
3,130514,2013-10-26,MAC001544,13.980,15.87,225,10.34,0.32,7.05,996.66,15.87,1,11.18,0.81,11.70,15.87,1.0,11.70,11.66,15.87,11.66,0.76,0,0,0,0,0,1,0,3,46,0,23.422,10,0,2013
4,1508327,2013-03-15,MAC000655,6.663,8.27,207,2.72,0.65,5.82,1007.60,4.56,1,12.36,0.81,1.78,4.56,2.0,6.36,2.99,8.27,-0.32,0.12,0,0,0,0,0,1,0,4,54,0,7.472,3,1,2013


#First without any feature scaling or Validation 


## Split data to traing and testing

In [9]:
from sklearn.model_selection import train_test_split

X = new_data.iloc[:, 4:-1].values
y = new_data.iloc[:,3].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=9)

In [10]:
from sklearn.ensemble import RandomForestRegressor

#regressor = RandomForestRegressor(n_estimators=8000,random_state= 42)
regressor = RandomForestRegressor(n_estimators=1000,random_state= 9)

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [13]:
from sklearn import metrics
import numpy as np

rrmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) / np.mean(y_test)
print('Relative root mean squared error percentage:', rrmse*100)


Relative root mean squared error percentage: 38.86752271621483


In [15]:
from sklearn.ensemble import ExtraTreesRegressor

regressor2 = ExtraTreesRegressor(n_estimators=1000, random_state=9)
regressor2.fit(X_train, y_train)
y_pred2 = regressor2.predict(X_test)

In [16]:
rrmse2 = np.sqrt(metrics.mean_squared_error(y_test, y_pred2)) / np.mean(y_test)
print('Relative root mean squared error percentage:', rrmse2*100)

Relative root mean squared error percentage: 39.83535717282509


#**Performing Standard Scaling**

In [23]:
new_data = pd.read_csv(path + "/sample_data_.csv")


In [24]:
new_data.head()

,Unnamed: 0,day,LCLid,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season,year
0,2175289,2012-07-19,MAC001497,4.364,17.77,340,11.44,0.54,1.52,1016.21,17.77,1,12.18,0.79,11.83,17.77,5.0,11.83,13.12,17.77,13.12,0.04,0,0,0,1,0,0,0,4,80,0,3.952,7,2,2012
1,2203521,2013-10-03,MAC003176,10.340,20.82,208,15.39,0.62,4.04,1009.01,20.84,1,11.68,0.87,13.67,20.84,1.0,13.67,15.23,20.82,15.23,0.98,0,0,0,1,0,0,0,4,81,0,13.006,10,0,2013
2,2158078,2012-09-07,MAC001679,4.236,27.12,215,9.27,0.14,1.39,1020.60,27.12,1,12.36,0.62,11.32,27.12,4.0,11.32,10.41,27.12,10.41,0.75,0,0,0,1,0,0,0,4,78,0,3.550,9,0,2012
3,130514,2013-10-26,MAC001544,13.980,15.87,225,10.34,0.32,7.05,996.66,15.87,1,11.18,0.81,11.70,15.87,1.0,11.70,11.66,15.87,11.66,0.76,0,0,0,0,0,1,0,3,46,0,23.422,10,0,2013
4,1508327,2013-03-15,MAC000655,6.663,8.27,207,2.72,0.65,5.82,1007.60,4.56,1,12.36,0.81,1.78,4.56,2.0,6.36,2.99,8.27,-0.32,0.12,0,0,0,0,0,1,0,4,54,0,7.472,3,1,2013


In [25]:
new_data = new_data.drop(['Unnamed: 0','day','LCLid','year'], axis = 1)

In [28]:
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()
scaled_data  = pd.DataFrame(scaler.fit_transform(new_data),columns = new_data.columns)





In [29]:
scaled_data.head()

,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,apparentTemperatureMin,moonPhase,icon_clear-day,icon_cloudy,icon_fog,icon_partly-cloudy-day,icon_partly-cloudy-night,icon_wind,stdorToU,Acorn_grouped,file,holiday_ind,last_day_sum,month,season
0,-0.617481,0.548764,1.617234,0.912536,0.312876,-1.224871,0.305502,0.589072,0.139076,0.373446,0.159506,0.829259,0.576512,1.154807,0.735127,1.041087,0.559614,1.083307,-1.603755,-0.077367,-0.105825,-0.285158,0.604963,-0.170037,-0.416123,-0.492371,1.372687,0.777744,-0.152147,-0.657642,0.124664,0.447214
1,0.024765,1.017587,0.169658,1.696097,0.731430,0.270816,-0.388285,0.988979,0.139076,0.158081,0.973442,1.123559,0.985902,-0.951857,1.101128,1.458296,1.022310,1.418813,1.657951,-0.077367,-0.105825,-0.285158,0.604963,-0.170037,-0.416123,-0.492371,1.372687,0.808509,-0.152147,0.301646,0.994819,-1.341641
2,-0.631238,1.985976,0.246423,0.482073,-1.779895,-1.302029,0.728519,1.807029,0.139076,0.450978,-1.570108,0.747686,1.823354,0.628141,0.633680,0.505239,1.978043,0.652396,0.859874,-0.077367,-0.105825,-0.285158,0.604963,-0.170037,-0.416123,-0.492371,1.372687,0.716213,-0.152147,-0.700234,0.704768,-1.341641
3,0.415959,0.256710,0.356088,0.694329,-0.838148,2.057330,-1.578322,0.341572,0.139076,-0.057284,0.362990,0.808466,0.323143,-0.951857,0.709268,0.752401,0.271377,0.851156,0.894573,-0.077367,-0.105825,-0.285158,-1.652994,-0.170037,2.403137,-0.492371,0.097282,-0.268274,-0.152147,1.405240,0.994819,-1.341641
4,-0.370406,-0.911505,0.158692,-0.817250,0.888388,1.327293,-0.524151,-1.131701,0.139076,0.450978,0.362990,-0.778196,-1.185070,-0.425191,-0.352933,-0.961915,-0.881571,-1.053755,-1.326163,-0.077367,-0.105825,-0.285158,-1.652994,-0.170037,2.403137,-0.492371,1.372687,-0.022153,-0.152147,-0.284691,-1.035542,-0.447214


In [38]:
from sklearn.model_selection import train_test_split

X = scaled_data.iloc[:,1:].values
y = scaled_data.iloc[:,0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=9)

In [39]:
from sklearn.ensemble import RandomForestRegressor

#regressor = RandomForestRegressor(n_estimators=8000,random_state= 42)
regressor = RandomForestRegressor(n_estimators=1000,random_state= 9)

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [40]:
rrmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) / np.mean(y_test)
print('Relative root mean squared error percentage:', rrmse*100)


Relative root mean squared error percentage: -2108.432823542845


In [41]:
# from sklearn.ensemble import ExtraTreesRegressor

#regressor = RandomForestRegressor(n_estimators=8000,random_state= 42)
regressor2 = ExtraTreesRegressor(n_estimators=1000,random_state= 9)

regressor2.fit(X_train, y_train)
y_pred = regressor2.predict(X_test)


In [42]:

rrmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) / np.mean(y_test)
print('Relative root mean squared error percentage:', rrmse*100)

Relative root mean squared error percentage: -2162.6439968575114
